<a href="https://colab.research.google.com/github/Suhit97/resnet-50-transfer-learning/blob/main/Tensorflow_transfer_learning_resnet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
%matplotlib inline
print("tensorflow version:", tf.__version__)

tensorflow version: 2.4.0


In [2]:
BATCH_SIZE = 32
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [3]:
(training_images, training_labels) , (validation_images, validation_labels) = tf.keras.datasets.cifar10.load_data()

170500096/170498071 [==============================] - 11s 0us/step


In [4]:
def process_input(images):
    img = images.astype('float')
    img = tf.keras.applications.resnet50.preprocess_input(img)

    return img

In [5]:
X_train = process_input(training_images)
X_val = process_input(validation_images)

In [6]:
print("Shape of X_train : ",X_train.shape)
print("Shape of training_labels : ",training_labels.shape)
print("Shape of X_val : ",X_val.shape)
print("Shape of validation_labels : ",validation_labels.shape)

Shape of X_train :  (50000, 32, 32, 3)
Shape of training_labels :  (50000, 1)
Shape of X_val :  (10000, 32, 32, 3)
Shape of validation_labels :  (10000, 1)


In [7]:
def predefine_model(inputs):
    resnet = tf.keras.applications.resnet.ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))
    x = resnet(inputs)
    return x

In [8]:
def outputlayer(inputs):

    x = predefine_model(inputs)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(1024, activation = 'relu')(x)
    x = tf.keras.layers.Dense(512, activation = 'relu')(x)
    x = tf.keras.layers.Dense(10, activation = 'softmax', name="classification" )(x)

    return x


In [9]:
def get_model():
    Input = tf.keras.layers.Input(shape=(32 , 32 , 3))
    x     = tf.keras.layers.UpSampling2D(size=(7,7))(Input) # as 7x32 = 128 which is imagenet ideal i/p
    x     = outputlayer(x)

    model = tf.keras.models.Model(inputs = Input , outputs = x)
    model.compile(loss = tf.keras.losses.sparse_categorical_crossentropy , optimizer = 'sgd' , metrics = ['acc'])
    return model



In [10]:
model = get_model()
model.summary()

94773248/94765736 [==============================] - 1s 0us/step
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
up_sampling2d (UpSampling2D) (None, 224, 224, 3)       0         
_________________________________________________________________
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              2098176   
______________________________________________________________

In [11]:
epoch = 4
his = model.fit(X_train , training_labels ,
                epochs=epoch , batch_size=32 ,
                validation_data=(X_val , validation_labels) ,
                validation_batch_size=32)

Epoch 1/4
1563/1563 [==============================] - 292s 181ms/step - loss: 0.6107 - acc: 0.7966 - val_loss: 0.2274 - val_acc: 0.9212
Epoch 2/4
1563/1563 [==============================] - 282s 180ms/step - loss: 0.0950 - acc: 0.9701 - val_loss: 0.1481 - val_acc: 0.9505
Epoch 3/4
1563/1563 [==============================] - 282s 180ms/step - loss: 0.0376 - acc: 0.9882 - val_loss: 0.1687 - val_acc: 0.9490
Epoch 4/4
1563/1563 [==============================] - 282s 180ms/step - loss: 0.0202 - acc: 0.9946 - val_loss: 0.1677 - val_acc: 0.9541
